In [ ]:
about_the_run = "About the run: Getting the plots"
run, load_run_number = 2, 51             #  0:"single_run"  1:"keras_tuner" 2:"load_best_model"
colab = 0
model_choice = "tf"
percentage = 100
epochs = 10
batch_size=32
limit_of_nans_in_a_timestep = 120
percentage_of_data_in_summer_months = [0.4]  # Desired number of zero values
divide_latitude_in_these_many_parts = 5  #latitude has 10 values
divide_longitude_in_these_many_parts = 6 # longitude has 14 values

# Configurations

In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
import plotly.express as px
import math
import time
from sklearn.linear_model import LinearRegression
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import optimizers
from kerastuner.tuners import RandomSearch
import logging
import os
import sys
from contextlib import contextmanager


In [ ]:
# for the WSL conda environment
# 1. python kernel is named wslminiconda3 (Python 3.11.14)
# 2. the environment is named base and is in the directory /home/arhab/wslminiconda3
# 3. Do not use the python kernel named base

In [ ]:
def get_next_folder_name(base_folder):
    run_number = 1
    while True:
        folder_name = os.path.join(base_folder, f"run_{run_number}")
        if not os.path.exists(folder_name):
            return folder_name
        run_number += 1

# Base folder directory
base_directory = r"D:\thesis_data\notebooks\model_runs"

# Get the next folder name
folder_name = get_next_folder_name(base_directory)

# Create the folder
os.makedirs(folder_name, exist_ok=True)
run_number = f"run_{folder_name.split('_')[3]}"
print(run_number)

In [ ]:
# Set up logging and save the log file in the folder
log_file = os.path.join(folder_name, 'log.txt')
logger = logging.getLogger(folder_name)
logger.setLevel(logging.INFO)

# Create a file handler and set the formatter
file_handler = logging.FileHandler(log_file)
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
file_handler.setFormatter(formatter)

# Add the file handler to the logger
logger.addHandler(file_handler)

# Create a stream handler to display log messages in Jupyter Notebook console
stream_handler = logging.StreamHandler()
stream_handler.setFormatter(formatter)
logger.addHandler(stream_handler)

# Log information
logger.info("Logging information check")
logger.info(run_number)
logger.info(about_the_run)

In [ ]:

logger.info(f"run = {run}   =>  0:single_run  1:keras_tuner 2:load_best_model")
logger.info(f"model_choice ={model_choice}")
logger.info(f"percentage = {percentage}")
logger.info(f"epochs = {epochs}")
logger.info(f"batch_size = {batch_size}")
logger.info(f"limit_of_nans_in_a_timestep = {limit_of_nans_in_a_timestep}")
logger.info(f"percentage_of_data_in_summer_months = {percentage_of_data_in_summer_months}")
logger.info(f"divide_latitude_in_these_many_parts = {divide_latitude_in_these_many_parts}")  
logger.info(f"divide_longitude_in_these_many_parts = {divide_longitude_in_these_many_parts}") 


In [ ]:
import contextlib
import sys

@contextlib.contextmanager
def stdout_redirected(new_stdout):
    save_stdout = sys.stdout
    sys.stdout = new_stdout
    try:
        yield None
    finally:
        sys.stdout = save_stdout


In [ ]:
if colab == 1:
    # !pip install cftime
    from google.colab import drive
    drive.mount('/content/drive')
    import cftime
    %run "/content/drive/My Drive/Colab Notebooks/main/functions.ipynb"
else: 
    %run "functions.ipynb"

# Data Loading

In [ ]:
# Step 1: Load NDSI Labels
if colab !=1:
  ndsi_ds = xr.open_mfdataset(r".\cropped_data\label.nc")

if colab == 1:
  ndsi_ds = xr.open_mfdataset(f'/content/drive/My Drive/Colab Notebooks/cropped_data/label.nc')

filtered_dates = get_filtered_dates_for_ndsi(limit_of_nans_in_a_timestep= limit_of_nans_in_a_timestep)
ndsi_ds = ndsi_ds.sel(time=filtered_dates)
selected_dates = get_dates(ndsi_ds)
ndsi_labels = ndsi_ds['NDSI_Snow_Cover'].values
ndsi_ds = ndsi_ds.interpolate_na(dim='lon', method='linear',  max_gap=4, use_coordinate=False)
ndsi_ds = ndsi_ds.interpolate_na(dim='lat', method='linear', max_gap=3, use_coordinate=False)

ndsi_ds['time'] = xr.DataArray(ndsi_ds['time'].values.astype('datetime64[ns]'), dims='time', attrs=ndsi_ds['time'].attrs)
ndsi_ds.close()

In [ ]:
ndsi_ds

In [ ]:
# Step 2: Load Data
# Load climate variables
climate_vars = ["tas", "pr", "hurs", "psl",  "rsds", "sfcWind"]
data = []

for parameter in climate_vars:
    scenario = "observational"

    if colab == 0:
      ds = xr.open_mfdataset(rf'.\cropped_data\{scenario}\{parameter}_{scenario}.nc') #fixxx
    if colab == 1:
      ds = xr.open_mfdataset(f'/content/drive/My Drive/Colab Notebooks/cropped_data/{scenario}/{parameter}_{scenario}.nc')


    # ds = ds.sel(time=slice("2001-01-01", "2018-31-02"))
    
    ds = ds.sel(time=selected_dates)
    data.append(ds[parameter].values)
    ds.close()

data = np.array(data)

# Conversion to pandas from xarray

In [ ]:
# Step 5: Flatten Data
n_time_steps = data[0].shape[0]
n_lat, n_lon = data[0].shape[1], data[0].shape[2]
logger.info("for feature")
logger.info(f"{n_time_steps} , {n_lat}, {n_lon}")

n_time_steps = ndsi_labels.shape[0]
n_lat, n_lon = ndsi_labels.shape[1], ndsi_labels.shape[2]
logger.info("for label")
logger.info(f"{n_time_steps} , {n_lat}, {n_lon}")


In [ ]:
temp0 = to_array(data, 0)
temp1 = to_array(data, 1)
temp2 = to_array(data, 2)
temp3 = to_array(data, 3)
temp4 = to_array(data, 4)
temp5 = to_array(data, 5)

ndsi_array = to_array_ndsi(ndsi_labels)

In [ ]:
dates = ds["time"].values
lats = ndsi_ds["lat"].values
lons = ndsi_ds["lon"].values

#********** Dates *************
dates_array_to_append = []
for a in range(len(dates)):
    x = dates[a]
    for b in range(140):
        dates_array_to_append.append(x)
dates_array_to_append = np.array(dates_array_to_append)

#********** Latitude *************
lats_array_to_append = []
for c in range(len(dates)):
    for a in range(len(lats)):
        x = lats[a]
        for b in range(len(lons)):
            lats_array_to_append.append(x)
lats_array_to_append = np.array(lats_array_to_append)

#********** Longitude *************
lons_array_to_append = []
for b in range(len(dates)*len(lats)):
    for a in range(len(lons)):
        x = lons[a]
        lons_array_to_append.append(x)

lons_array_to_append = np.array(lons_array_to_append)


logger.info(f"Length of Dates: {len(dates_array_to_append)}")
logger.info(f"Length of Latitude: {len(lats_array_to_append)}")
logger.info(f"Length of Longitude: {len(lons_array_to_append)}")

# Data cleaning, feature engineering with pandas

In [ ]:

dict_temp = {
    "Date": dates_array_to_append,
    "Latitude": lats_array_to_append,
    "Longitude": lons_array_to_append,
    climate_vars[0]: temp0,
    climate_vars[1]: temp1,
    climate_vars[2]: temp2,
    climate_vars[3]: temp3,
    climate_vars[4]: temp4,
    climate_vars[5]: temp5,
    "ndsi1": ndsi_array
}


# Create the DataFrame
df = pd.DataFrame(dict_temp)
df['Latitude'] = df['Latitude'].round(2)
df['Longitude'] = df['Longitude'].round(2)
df, drop_nan = df.dropna(ignore_index=False), "yes"


In [ ]:
# df = df.reset_index()
df["Month"] = df['Date'].dt.month
df['month_sin'] = np.sin(2 * np.pi * df['Month']/12)
df['month_cos'] = np.cos(2 * np.pi * df['Month']/12)

df['week_number'] = df["Date"].dt.isocalendar().week

In [ ]:
# # Original one-hot encoding
# lat_one_hot_encoded = pd.get_dummies(df['Latitude'], prefix='Latitude', dtype=int)
# lon_one_hot_encoded = pd.get_dummies(df['Longitude'], prefix='Longitude', dtype=int)

# df = pd.concat([df, lat_one_hot_encoded, lon_one_hot_encoded], axis=1)


In [ ]:
# # One-hot encoding
# #Latitude
# df['Lat_46_to_46d36'] = (df['Latitude'].between(46, 46.36)).astype(int)
# df['Lat_46d37_to_46d69'] = (df['Latitude'].between(46.36, 46.70)).astype(int)
# df['Lat_46d8_to_47d14'] = (df['Latitude'].between(46.70, 47.15)).astype(int)

# #Longitude
# df['Lon_10_to_10d67'] = (df['Longitude'].between(10, 10.67)).astype(int)
# df['Lon_10d67_to_11d32'] = (df['Longitude'].between(10.67, 11.32)).astype(int)
# df['Lon_11d32_to_11d81'] = (df['Longitude'].between(11.33, 11.82)).astype(int)
# df['Lon_11d81_to_12d29'] = (df['Longitude'].between(11.83, 12.29)).astype(int)


In [ ]:
lats = np.unique(df.Latitude.values)
lons = np.unique(df.Longitude.values)

latitude_bins = divide_range(lats[0], lats[-1], divide_latitude_in_these_many_parts)   #10 values 
longitude_bins = divide_range(lons[0], lons[-1], divide_longitude_in_these_many_parts) #14 values

df['Latitude_Group'] = pd.cut(df['Latitude'], bins=latitude_bins, labels=False)
df['Longitude_Group'] = pd.cut(df['Longitude'], bins=longitude_bins, labels=False)

# Apply one-hot encoding
latitude_dummies = pd.get_dummies(df['Latitude_Group'], prefix='Latitude', dtype=int)
longitude_dummies = pd.get_dummies(df['Longitude_Group'], prefix='Longitude', dtype=int)

# Concatenate one-hot encoded columns with the original DataFrame
df = pd.concat([df, latitude_dummies, longitude_dummies], axis=1)
temp = df
df = df.drop(["Latitude_Group",	"Longitude_Group"], axis =1)

temp = temp[temp["Date"] == temp.Date[1]]
grouped_latitudes = temp.groupby('Latitude_Group')['Latitude'].unique().to_dict()
logger.info("Latitude Group")
logger.info("{\n" + ",\n".join(f" {key}: {value}" for key, value in grouped_latitudes.items()) + "\n}")

grouped_longitudes = temp.groupby('Longitude_Group')['Longitude'].unique().to_dict()
logger.info("Longitude Group")
logger.info("{\n" + ",\n".join(f" {key}: {value}" for key, value in grouped_longitudes.items()) + "\n}")


In [ ]:
#bookmark
print(df.columns)

In [ ]:
df["ndsi"] = df["ndsi1"]
df= df.drop(["Month", "week_number", "ndsi1"],axis =1)

In [ ]:
parameter_array = ["tas", "pr", "hurs", "psl",  "rsds", "sfcWind"]
min_max_df = pd.read_csv("./min_max_of_all_parameters.csv")
for parameter in parameter_array:
    min_val, max_val =  get_min_max(min_max_df, parameter)
    df[parameter] = (df[parameter] - min_val) / (max_val - min_val)

df['ndsi'] = df['ndsi'].apply(lambda x: x / 100 if not pd.isnull(x) else x)

In [ ]:
df_insurance = df
df_insurance

In [ ]:

df_temp = df_insurance
# Plot histogram for df
fig1 = px.histogram(df_temp, x='ndsi', nbins=100)
fig1.update_layout(
    xaxis_title='Value',
    yaxis_title='Frequency',
)

fig1


In [ ]:
# b_array = [0.1, 0.15, 0.2, 0.25, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.99]  # Desired number of zero values
zero_values_array = []
b_array = percentage_of_data_in_summer_months
for b in b_array:
    
    df = df_insurance
    df["Month"] = df['Date'].dt.month
    df_to_delete = df
    df = df[df['Month'].isin([1, 2, 3, 12])]
    logger.info(f"Number of datapoints in Original df: {len(df_to_delete)}")

    # Step 6: Display the final value of b
    logger.info(f"Value of b: {b}")


    # Now you can use this final value of b to sample your DataFrame
    final_sampled_data = pd.concat([df_to_delete[(df_to_delete['Date'].dt.month == month)].sample(frac=b) for month in range(4, 12)])
    df = pd.concat([final_sampled_data, df], ignore_index=False)
    df = df.sort_index()

    number_of_zeroes = np.sum((df['ndsi'] >= -0.005) & (df['ndsi'] <= 0.005))
    zero_values_array.append(number_of_zeroes)
    logger.info(f"Total number of zeroes: {number_of_zeroes}")


In [ ]:
# # number_of_points_at_the_peak = np.sum((df['ndsi'] >= 0.675) & (df['ndsi'] <= 0.685))
# # a = multiplication_factor*number_of_points_at_the_peak - 2500 
# a = desired_number_of_zeroes - 2500
# tolerance = 0.15 * a 

# df["Month"] = df['Date'].dt.month
# df_to_delete = df
# df = df[df['Month'].isin([1, 2, 3, 12])]

# logger.info(f"Tolerance:  {tolerance}")
# logger.info(f"Final range of near zeores: {a + 2500 - tolerance} - {a + 2500 + tolerance}")

# b = 0.01  # Initial percentage to sample
# while True:
#     zero_values_array = []
#     logger.info(f"{b}")
    
#     # Step 2: Iterate through each month
#     for month in range(4, 12):  # Months from April to November
#         # Step 3: Sample rows and calculate the number of near zero values
#         sampled_data = df_to_delete[(df_to_delete['Date'].dt.month == month)].sample(frac=b)
#         zero_values_array.append(np.sum((sampled_data['ndsi'] >= -0.005) & (sampled_data['ndsi'] <= 0.005)))
        
#     # Step 4: Check the sum of zero values
#     total_zero_values = sum(zero_values_array)
#     logger.info(f"{total_zero_values}")

#     # Step 5: Adjust the percentage (b)
#     if total_zero_values > a + tolerance:
#         b -= 0.01
#     elif total_zero_values < a - tolerance:
#         b += 0.01
#     else:
#         break  # Convergence reached

# b = round(b,2)
# # Step 6: Display the final value of b
# logger.info(f"Final value of b: {b}")
# logger.info(f"Total zero values: {total_zero_values+ (percentage*2500)}")

# # Now you can use this final value of b to sample your DataFrame
# final_sampled_data = pd.concat([df_to_delete[(df_to_delete['Date'].dt.month == month)].sample(frac=b) for month in range(4, 12)])
# df = pd.concat([final_sampled_data, df], ignore_index=False)
# df = df.sort_index()

# # the old code for the sampling has been moved to less_imp/rough

### Plotting of per month data (should be kept folded to keep everything compact)

In [ ]:
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objs as go

# Plot histogram for df
fig1 = px.histogram(df, x='ndsi', nbins=100)
fig1.update_layout(
    xaxis_title='Value',
    yaxis_title='Frequency',
)

# Plot histogram for df_to_delete
fig2 = px.histogram(df_to_delete, x='ndsi', nbins=100)
fig2.update_layout(
    xaxis_title='Value',
    yaxis_title='Frequency',
)

# Create subplots with two columns
fig = make_subplots(rows=1, cols=2, subplot_titles=('Before', 'After'))
fig.add_trace(fig1.data[0], row=1, col=2)
fig.add_trace(fig2.data[0], row=1, col=1)

fig.update_layout(
    title='Before and after zero sampling in mid-year months',
    width=1000,  # Total width of the combined plots
    height=400,  # Height of the combined plots
)

plotly_image_path = os.path.join(folder_name, 'before_after_zero_sampling.png')
fig.write_image(plotly_image_path)


fig.show()


In [ ]:
#bookmark
import pandas as pd
import matplotlib.pyplot as plt

# Assuming 'df_to_delete' is your DataFrame
# Convert 'Date' column to datetime
month_fraction_array_padded = generate_array(b)
df_to_delete['Date'] = pd.to_datetime(df_to_delete['Date'])

# Extract month from the 'Date' column
df_to_delete['Month'] = df_to_delete['Date'].dt.month
df['Month'] = df['Date'].dt.month

# Create a new figure and subplots
fig, axs = plt.subplots(4, 6, figsize=(20, 10))  # 4 rows, 8 columns for 12 months for both DataFrames


# row = (month - 1) // 3  # Calculate the row index for the subplot
#     col = (month - 1) % 3

# Plot histograms for each month in df
for month in range(1, 13):
    row = (month - 1) // 3  # Calculate the row index for the subplot
    col = (month - 1) % 3   # Calculate the column index for the subplot
    
    # Filter data for the current month in df_to_delete
    month_data_df_to_delete = df_to_delete[df_to_delete['Month'] == month]['ndsi']
    
    # Plot histogram in the corresponding subplot for df_to_delete
    axs[row, col].hist(month_data_df_to_delete, bins=20, color='salmon', alpha=0.7)
    axs[row, col].set_title(f'Month {month} (before)')
    axs[row, col].set_xlabel('NDSI')
    axs[row, col].set_ylabel('Frequency')


# Plot histograms for each month in df_to_delete on the right
for month in range(1, 13):
    row = (month - 1) // 3  # Calculate the row index for the subplot
    col = (month - 1) % 3 + 3  # Shift to the right by 4 columns
    
    # Filter data for the current month in df
    month_data_df = df[df['Month'] == month]['ndsi']
    
    # Plot histogram in the corresponding subplot for df
    axs[row, col].hist(month_data_df, bins=20, color='skyblue', alpha=0.7)
    axs[row, col].set_title(f'Month {month} (after) | Fraction = {month_fraction_array_padded[month-1]}')
    axs[row, col].set_xlabel('NDSI')
    axs[row, col].set_ylabel('Frequency')
    
    
# # Hide empty subplots
# for i in range(12, 16):
#     fig.delaxes(axs[3, i])

df = df.drop(["Month"], axis=1)
# Adjust layout and display the subplots
plt.tight_layout()

matplotlib_image_path = os.path.join(folder_name, 'each_month_zero_cleaning.png')
plt.savefig(matplotlib_image_path)


plt.show()


In [ ]:
if percentage == 100:
    df.to_csv("./complete_df.csv")

# Data splitting

In [ ]:
# percentage = 100
number_of_rows = int(percentage/100*len(df))
logger.info(f"Number of rows: {number_of_rows}/{len(df)}")

In [ ]:
df_reduced = df.sample(frac=percentage/100)
df_reduced = df_reduced.sort_index()

In [ ]:
# test= df_reduced[df_reduced['Date'] >= "2018-12-30"]
# working_dataset = df_reduced.drop(test.index)
# working_dataset = working_dataset.sort_index()

working_dataset = df_reduced

In [ ]:
val = working_dataset.sample(frac=0.2)
val = val.sort_index()

train = working_dataset.drop(val.index)
train = train.sort_index()

In [ ]:
# X = df.iloc[:number_of_rows, 1:-1].values
# y = df.iloc[:number_of_rows, -1].values

# Step 7: Split Data
X_train, X_val = train.drop(["Date", "ndsi", "Latitude", "Longitude"], axis=1).values, val.drop(["Date", "ndsi", "Latitude", "Longitude"], axis=1).values
y_train, y_val = train["ndsi"].values, val["ndsi"].values

In [ ]:
# X_test, y_test = test.drop(["Date", "ndsi","Latitude", "Longitude"], axis=1).values, test["ndsi"].values

In [ ]:
temp_shape = X_train.shape[1]
# Extracting features and labels
X_train = X_train.reshape(-1, 1, temp_shape)  # Reshaping to (146246, 1, 32)
y_train = y_train.reshape(-1, 1)  # Assuming the label is in column 35

X_val = X_val.reshape(-1, 1, temp_shape)  # Reshaping to (146246, 1, 32)
y_val = y_val.reshape(-1, 1)  # Assuming the label is in column 35

# X_test = X_test.reshape(-1, 1, temp_shape)  # Reshaping to (146246, 1, 32)
# y_test = y_test.reshape(-1, 1)  # Assuming the label is in column 35


# logger.infoing shapes
logger.info(f"y_train shape is: {y_train.shape}")
logger.info(f"X_train shape is: {X_train.shape}")

logger.info(f"y_val shape is: {y_val.shape}")
logger.info(f"X_val shape is: {X_val.shape}")

# logger.info(f"y_test shape is: {y_test.shape}")
# logger.info(f"X_test shape is: {X_test.shape}")


# Model Architecture

## Machine learning stuff, mainly folded

In [ ]:
if model_choice=="ml":
    # Step 8: Train Random Forest Model
    rf_model = RandomForestRegressor(n_estimators=100)

    start_time = time.time()
    rf_model.fit(X_train, y_train)
    end_time = time.time()
    execution_time_fitting = end_time - start_time
    logger.info(f"Time taken to fit {execution_time_fitting/60} min")
    logger.info("Fitting done")

    # Step 9: Model historical
    start_time = time.time()
    y_val_pred = rf_model.predict(X_val)
    end_time = time.time()
    # y_test_pred = rf_model.predict(X_test)
    execution_time_prediciting = end_time - start_time

    logger.info(f"Time taken to predict {execution_time_prediciting}")


## Deep learning stuff, the main thing 

In [ ]:
if model_choice=="tf":
    import tensorflow as tf
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Dense, Dropout, LSTM, SimpleRNN
    from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
    from tensorflow.keras import optimizers

    # Check for GPU availability
    if tf.config.list_physical_devices('GPU'):
        logger.info('GPU found. Running on GPU.')
    else:
        logger.info('No GPU found. Running on CPU.')

    # Specify GPU device if available
    physical_devices = tf.config.list_physical_devices('GPU')
    if len(physical_devices) > 0:
        tf.config.experimental.set_memory_growth(physical_devices[0], True)

    # Convert data to TensorFlow tensors
    X_train_tensor = tf.convert_to_tensor(X_train, dtype=tf.float32)
    y_train_tensor = tf.convert_to_tensor(y_train, dtype=tf.float32)
    
    X_val_tensor = tf.convert_to_tensor(X_val, dtype=tf.float32)
    y_val_tensor = tf.convert_to_tensor(y_val, dtype=tf.float32)
    
    # X_test_tensor = tf.convert_to_tensor(X_test, dtype=tf.float32)
    # y_test_tensor = tf.convert_to_tensor(y_test, dtype=tf.float32)


In [ ]:
if model_choice=="tf" and run ==0:
    # Define the model
    model = Sequential()
    model.add(LSTM(units=180, input_shape=(1,temp_shape), return_sequences=True, activation='relu'))
    model.add(LSTM(50, return_sequences=True))
    model.add(LSTM(10, return_sequences=True))
    model.add(LSTM(5, return_sequences=False))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1, activation='linear'))  # Assuming NDSI is a continuous variable

    # Compile the model
    model.compile(optimizer="adam", loss='mean_squared_error', metrics=['mae'])

    model_save_path = os.path.join(folder_name, 'model.h5')
    callbacks = [
        EarlyStopping(monitor='val_loss', patience=4),
        ModelCheckpoint(model_save_path, monitor='val_loss', save_best_only=True, mode='min')
    ]

    with open(log_file, 'a') as f:
        with stdout_redirected(f):
            model.summary()
            
            # Train the model on GPU
            start_time = time.time()
            history = model.fit(X_train_tensor, y_train_tensor, epochs=epochs, batch_size=batch_size, callbacks=callbacks, validation_data=(X_val_tensor, y_val_tensor))
            end_time = time.time()
            execution_time_fitting = end_time - start_time

            # Evaluate the model on GPU
            loss, mae = model.evaluate(X_val_tensor, y_val_tensor)
            logger.info(f'Mean absolute Error on val Set: {mae}')

    # Make predictions on GPU
    y_val_pred = model.predict(X_val_tensor)
    # y_test_pred = model.predict(X_test_tensor)

    remove_eta_lines(run_number)

In [ ]:
from tensorflow.keras.losses import MeanSquaredError, MeanAbsoluteError, Huber
if model_choice == "tf" and run == 1:

    # Define a function to build the model with hyperparameters
    def build_model(hp):
        model = Sequential()
        model.add(LSTM(units=hp.Int('units_1', min_value=50, max_value=200, step=10), input_shape=(1, temp_shape), return_sequences=True, activation=hp.Choice('activation_1', ['sigmoid', 'tanh', 'relu'])))
        model.add(LSTM(units=hp.Int('units_2', min_value=20, max_value=100, step=10), return_sequences=True, activation=hp.Choice('activation_2', ['sigmoid', 'tanh', 'relu'])))
        model.add(LSTM(units=hp.Int('units_3', min_value=10, max_value=50, step=10), return_sequences=True, activation=hp.Choice('activation_3', ['sigmoid', 'tanh', 'relu'])))
        model.add(LSTM(units=hp.Int('units_4', min_value=5, max_value=30, step=5), return_sequences=False, activation=hp.Choice('activation_4', ['sigmoid', 'tanh', 'relu'])))
        model.add(Dense(units=hp.Int('units_5', min_value=10, max_value=100, step=10), activation='relu'))
        model.add(Dense(1, activation='linear'))

        optimizer_choice = hp.Choice('optimizer', ['adam', 'rmsprop', 'sgd'])  # Define optimizers for regression
        if optimizer_choice == 'adam':
            optimizer = optimizers.Adam(learning_rate=hp.Choice('learning_rate_adam', values=[1e-2, 1e-3, 1e-4]))
        elif optimizer_choice == 'rmsprop':
            optimizer = optimizers.RMSprop(learning_rate=hp.Choice('learning_rate_rmsprop', values=[1e-2, 1e-3, 1e-4]))
        else:
            optimizer = optimizers.SGD(learning_rate=hp.Choice('learning_rate_sgd', values=[1e-2, 1e-3, 1e-4]))

        loss_choice = hp.Choice('loss', ['mse', 'mae', 'huber_loss'])  # Define loss functions
        if loss_choice == 'mse':
            loss = MeanSquaredError()
        elif loss_choice == 'mae':
            loss = MeanAbsoluteError()
        else:
            loss = Huber()

        model.compile(optimizer=optimizer, loss=loss, metrics=['mae'])
        return model

    # Instantiate the tuner for hyperparameter search
    tuner = RandomSearch(
        build_model,
        objective='val_loss',
        max_trials=100,  # Number of hyperparameter combinations to try
        executions_per_trial=1,
        directory=folder_name,  # Directory to store the results
        project_name='regression_optimizers_tuner_2'
    )

    with open(log_file, 'a') as f:
        with stdout_redirected(f):
            # Search for the best hyperparameter configuration
            tuner.search(X_train_tensor, y_train_tensor, epochs=10, batch_size = 128, validation_data=(X_val_tensor, y_val_tensor))

    remove_eta_lines(run_number)


In [ ]:
if model_choice=="tf" and run ==1:
    # Get the best hyperparameters and build the final model
    best_hp = tuner.get_best_hyperparameters(num_trials=1)[0]
    model = tuner.hypermodel.build(best_hp)

    # Train the best model
    history = model.fit(X_train_tensor, y_train_tensor, epochs=epochs, batch_size=batch_size, validation_data=(X_val_tensor, y_val_tensor), callbacks=[EarlyStopping(patience=20)])

    # Evaluate the best model
    loss, mae = model.evaluate(X_val_tensor, y_val_tensor)
    logger.info(f'Mean absolute Error on val Set: {mae}')

    # Make predictions using the best model
    y_val_pred = model.predict(X_val_tensor)
    # y_test_pred = best_model.predict(X_test_tensor)


In [ ]:
# run =2
# load_run_number = 51

In [ ]:
from keras.models import load_model
if model_choice=="tf" and run ==2:
    # Provide the directory path where checkpoint files or model information is stored

    # Load the model
    model = load_model(rf'D:\thesis_data\notebooks\model_runs\run_{load_run_number}\model.h5')

    # Evaluate the best model
    loss, mae = model.evaluate(X_val_tensor, y_val_tensor)
    logger.info(f'Mean absolute Error on val Set: {mae}')

    # Make predictions using the best model
    y_val_pred = model.predict(X_val_tensor)
    # y_test_pred = loaded_model.predict(X_test_tensor)


In [ ]:
if model_choice=="tf" and run!=2:
    # Plot the training and validation loss
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Mean Squared Error')
    plt.legend()
    
    matplotlib_image_path = os.path.join(folder_name, 'loss_vs_epoch_graph.png')
    plt.savefig(matplotlib_image_path)
    
    plt.show()

In [ ]:
import math
from sklearn.metrics import  mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

if model_choice == "tf":
    y_val_pred = y_val_pred.flatten()
    y_val = y_val.flatten()


mae = mean_absolute_error(y_val, y_val_pred)
r2 = r2_score(y_val, y_val_pred)
# nse_value = nse(y_val_pred, y_val)

mse_value = mean_squared_error(y_val,y_val_pred)
rmse_value = math.sqrt(mse_value)
# n_rmse_value = calculate_n_rmse(y_val, y_val_pred)

pbias_value = pbias(y_val, y_val_pred)

temp_percentage = round(number_of_rows*100/len(df),2)

In [ ]:
logger.info(f"{number_of_rows}/{len(df)} = {temp_percentage} % \n")
# logger.info(f"Time taken to fit {execution_time_fitting/60} min")
# logger.info(f"NSE value: {nse_value}")
logger.info(f'R2 Score: {r2}')
# logger.info(f'Self defined R2 Score: {self_defined_r2} \n')
logger.info(f"P-bias value: {pbias_value} \n")

# logger.info(f"MSE value: {mse_value}")
logger.info(f"RMSE value: {rmse_value}")
# logger.info(f"N-RMSE value: {n_rmse_value}")

# Plotting

In [ ]:
# y_val_pred = y_val_pred.flatten()
comparison_df = pd.DataFrame({
    "Date" : val["Date"].values,
    "Latitude" : val["Latitude"].values,
    "Longitude" : val["Longitude"].values,
    "Observed": y_val,
    "Predicted": y_val_pred.flatten()
    })
comparison_df['Date'] = pd.to_datetime(comparison_df['Date'])

df_image_path = os.path.join(folder_name, 'val_vs_predicted.csv')
comparison_df.to_csv(df_image_path)

In [ ]:
comparison_df, scale = comparison_df.sort_values(by='Date'), "Original without mean"
fig = px.scatter(comparison_df, x="Date" , y=["Predicted","Observed"])
fig.update_layout(title=f'Predicted vs Observed | {scale}', width = 1800, height = 400)

plotly_image_path = os.path.join(folder_name, 'Original_without_mean.png')
fig.write_image(plotly_image_path)

fig

In [ ]:
comparison_df_1 = comparison_df[(comparison_df['Latitude'] ==  46.58) & (comparison_df['Longitude'] == 11.15)]
comparison_df_1, scale = comparison_df_1.sort_values(by='Date'), "Centre without mean"
fig = px.line(comparison_df_1, x="Date" , y=["Predicted","Observed"])
fig.update_layout(title=f'Predicted vs Observed | {scale}', width = 1800, height = 400)

plotly_image_path = os.path.join(folder_name, 'Centre_without_mean.png')
fig.write_image(plotly_image_path)

fig

In [ ]:
# top left
temp_lat =  lats[-3]
temp_lon = lons[2]
comparison_df_1 = comparison_df[(comparison_df['Latitude'] ==  temp_lat) & (comparison_df['Longitude'] == temp_lon)]
comparison_df_1, scale = comparison_df_1.sort_values(by='Date'), "Top left"
fig = px.line(comparison_df_1, x="Date" , y = ["Predicted","Observed"])
fig.update_layout(title=f' | {scale}', width = 1800, height = 400)

plotly_image_path = os.path.join(folder_name, f"{scale}.png")
fig.write_image(plotly_image_path)

fig

In [ ]:
# top right
temp_lat =  lats[-2]
temp_lon = lons[-2]
comparison_df_1 = comparison_df[(comparison_df['Latitude'] ==  temp_lat) & (comparison_df['Longitude'] == temp_lon)]
comparison_df_1, scale = comparison_df_1.sort_values(by='Date'), "Top Right"
fig = px.line(comparison_df_1, x="Date" , y = ["Predicted","Observed"])
fig.update_layout(title=f'Predicted vs Observed| {scale}', width = 1800, height = 400)

plotly_image_path = os.path.join(folder_name, f"{scale}.png")
fig.write_image(plotly_image_path)

fig

In [ ]:
# bottom left
temp_lat =  lats[1]
temp_lon = lons[2]
comparison_df_1 = comparison_df[(comparison_df['Latitude'] ==  temp_lat) & (comparison_df['Longitude'] == temp_lon)]
comparison_df_1, scale = comparison_df_1.sort_values(by='Date'), "Bottom left"
fig = px.line(comparison_df_1, x="Date" ,y=["Predicted","Observed"])
fig.update_layout(title=f'Predicted vs Observed| {scale}', width = 1800, height = 400)

plotly_image_path = os.path.join(folder_name, f"{scale}.png")
fig.write_image(plotly_image_path)

fig

In [ ]:
# bottom right
temp_lat =  lats[1]
temp_lon = lons[-4]
comparison_df_1 = comparison_df[(comparison_df['Latitude'] ==  temp_lat) & (comparison_df['Longitude'] == temp_lon)]
comparison_df_1, scale = comparison_df_1.sort_values(by='Date'), "Bottom right"
fig = px.line(comparison_df_1, x="Date" , y=["Predicted","Observed"])
fig.update_layout(title=f'Predicted vs Observed | {scale}', width = 1800, height = 400)

plotly_image_path = os.path.join(folder_name, f"{scale}.png")
fig.write_image(plotly_image_path)

fig

In [ ]:
comparison_df['Date'], scale = comparison_df['Date'].dt.strftime('%d-%m-%Y'), "Average Daily"
comparison_df= comparison_df.groupby('Date')[["Observed", "Predicted"]].mean().reset_index()
comparison_df['Date'] = pd.to_datetime(comparison_df['Date'], format='%d-%m-%Y')
comparison_df = comparison_df.sort_values(by='Date')
# comparison_df

In [ ]:
fig = px.scatter(comparison_df, x="Date" , y=["Predicted","Observed"])
fig.update_layout(title=f'Predicted vs Observed | {scale}', width = 1800, height = 400)

plotly_image_path = os.path.join(folder_name, 'Average daily.png')
fig.write_image(plotly_image_path)
fig

In [ ]:
fig = px.line(comparison_df, x="Date" , y=["Predicted","Observed"])
fig.update_layout(title=f'Predicted vs Observed | {scale}', width = 1800, height = 400)

plotly_image_path = os.path.join(folder_name, 'Average daily (line).png')
fig.write_image(plotly_image_path)
fig

In [ ]:
comparison_df['Date'], scale = comparison_df['Date'].dt.strftime('%m-%Y'), "Average monthly"
comparison_df= comparison_df.groupby('Date')[["Observed", "Predicted"]].mean().reset_index()
comparison_df['Date'] = pd.to_datetime(comparison_df['Date'])
comparison_df = comparison_df.sort_values(by='Date')

In [ ]:
fig = px.line(comparison_df, x="Date" , y=["Predicted","Observed"])
fig.update_layout(title=f'Predicted vs Observed | {scale}', width = 1800, height = 400)


plotly_image_path = os.path.join(folder_name, 'Average monthly.png')
fig.write_image(plotly_image_path)

fig